In [1]:
%env CUDA_DEVICE_ORDER=PCI_BUS_ID
#%env CUDA_VISIBLE_DEVICES=3,4,5

env: CUDA_DEVICE_ORDER=PCI_BUS_ID


In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import numpy as np
from ganime.data.base import load_dataset
from ganime.model.vae.vae import VAE
from ganime.visualization.videos import display_videos
import tensorflow as tf
from tqdm import tqdm
tf.get_logger().setLevel('ERROR')

In [4]:
train_ds, test_ds, input_shape = load_dataset("moving_mnist_vae", "../../../data", batch_size=128)

2022-05-05 07:43:24.393161: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-05-05 07:43:27.034146: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 14252 MB memory:  -> device: 0, name: NVIDIA RTX A4000, pci bus id: 0000:01:00.0, compute capability: 8.6
2022-05-05 07:43:27.035106: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 14252 MB memory:  -> device: 1, name: NVIDIA RTX A4000, pci bus id: 0000:25:00.0, compute capability: 8.6
2022-05-05 07:43:27.035886: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:2 w

In [5]:
strategy = tf.distribute.MirroredStrategy()

In [6]:
with strategy.scope():
    model = VAE()

In [22]:
model.encoder.summary(120)

Model: "encoder"
________________________________________________________________________________________________________________________
 Layer (type)                          Output Shape               Param #       Connected to                            
 input_1 (InputLayer)                  [(None, 20, 64, 64, 1)]    0             []                                      
                                                                                                                        
 time_distributed (TimeDistributed)    (None, 20, 32, 32, 32)     320           ['input_1[0][0]']                       
                                                                                                                        
 time_distributed_1 (TimeDistributed)  (None, 20, 16, 16, 64)     18496         ['time_distributed[0][0]']              
                                                                                                                        
 time_distribut

In [23]:
model.decoder.summary(120)

Model: "decoder"
________________________________________________________________________________________________________________________
 Layer (type)                                         Output Shape                                    Param #           
 input_2 (InputLayer)                                 [(None, 20, 128)]                               0                 
                                                                                                                        
 time_distributed_6 (TimeDistributed)                 (None, 20, 8192)                                1056768           
                                                                                                                        
 time_distributed_7 (TimeDistributed)                 (None, 20, 16, 16, 32)                          0                 
                                                                                                                        
 time_distribut

In [9]:
with strategy.scope():
    model.compile(optimizer=tf.keras.optimizers.Adam())
history = model.fit(train_ds, epochs=100)

2022-05-05 07:43:29.531663: W tensorflow/core/grappler/optimizers/data/auto_shard.cc:766] AUTO sharding policy will apply DATA sharding policy as it failed to apply FILE sharding policy because of the following reason: Found an unshardable source dataset: name: "TensorSliceDataset/_1"
op: "TensorSliceDataset"
input: "Placeholder/_0"
attr {
  key: "Toutput_types"
  value {
    list {
      type: DT_UINT8
    }
  }
}
attr {
  key: "_cardinality"
  value {
    i: 9000
  }
}
attr {
  key: "is_files"
  value {
    b: false
  }
}
attr {
  key: "metadata"
  value {
    s: "\n\024TensorSliceDataset:0"
  }
}
attr {
  key: "output_shapes"
  value {
    list {
      shape {
        dim {
          size: 20
        }
        dim {
          size: 64
        }
        dim {
          size: 64
        }
        dim {
          size: 1
        }
      }
    }
  }
}



Epoch 1/100


2022-05-05 07:43:42.978010: I tensorflow/stream_executor/cuda/cuda_dnn.cc:366] Loaded cuDNN version 8101
2022-05-05 07:43:43.702341: I tensorflow/stream_executor/cuda/cuda_dnn.cc:366] Loaded cuDNN version 8101
2022-05-05 07:43:44.639908: I tensorflow/stream_executor/cuda/cuda_dnn.cc:366] Loaded cuDNN version 8101
2022-05-05 07:43:45.729320: I tensorflow/stream_executor/cuda/cuda_dnn.cc:366] Loaded cuDNN version 8101
2022-05-05 07:43:46.202900: I tensorflow/stream_executor/cuda/cuda_blas.cc:1774] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.
2022-05-05 07:43:46.390802: I tensorflow/stream_executor/cuda/cuda_dnn.cc:366] Loaded cuDNN version 8101
2022-05-05 07:43:47.596466: I tensorflow/stream_executor/cuda/cuda_dnn.cc:366] Loaded cuDNN version 8101
2022-05-05 07:43:48.698082: I tensorflow/stream_executor/cuda/cuda_dnn.cc:366] Loaded cuDNN version 8101
2022-05-05 07:43:49.342473: I tensorflow/stream_executor/cuda/cuda_dnn.cc:366] Loaded cuDNN ve

71/71 [==============================] - 26s 64ms/step - loss: 269.7760 - reconstruction_loss: 190.4296 - kl_loss: 11.7549
Epoch 2/100
71/71 [==============================] - 4s 56ms/step - loss: 123.9188 - reconstruction_loss: 110.8171 - kl_loss: 14.3833
Epoch 3/100
71/71 [==============================] - 4s 56ms/step - loss: 105.5721 - reconstruction_loss: 95.1970 - kl_loss: 17.8880
Epoch 4/100
71/71 [==============================] - 4s 55ms/step - loss: 99.8363 - reconstruction_loss: 87.6399 - kl_loss: 20.1244
Epoch 5/100
71/71 [==============================] - 4s 56ms/step - loss: 93.9029 - reconstruction_loss: 83.1203 - kl_loss: 21.5758
Epoch 6/100
71/71 [==============================] - 4s 55ms/step - loss: 90.0806 - reconstruction_loss: 78.6485 - kl_loss: 22.7381
Epoch 7/100
71/71 [==============================] - 4s 55ms/step - loss: 89.7224 - reconstruction_loss: 80.2033 - kl_loss: 23.0886
Epoch 8/100
71/71 [==============================] - 4s 55ms/step - loss: 90.2479 

In [10]:
train_ds_iterator = train_ds.as_numpy_iterator()
x = next(train_ds_iterator)

In [11]:
generated = model(x[0])

In [12]:
display_videos(generated, n_rows=1, n_cols=5)

In [13]:
display_videos(x[1], n_rows=1, n_cols=5)

In [14]:
test_ds_iterator = test_ds.as_numpy_iterator()
x = next(test_ds_iterator)

In [15]:
generated = model(x[0])

In [16]:
display_videos(generated, n_rows=1, n_cols=5)

In [17]:
display_videos(x[1], n_rows=1, n_cols=5)

In [18]:
model.save("B-VAE_LSTM_Longer")

2022-05-05 07:51:25.376958: W tensorflow/python/util/util.cc:368] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.
